## 00 Import Modules

In [1]:
!pip install -q --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

In [2]:
import os
import pathlib
import torch
import numpy as np
import textwrap

from random import randint
from itertools import zip_longest
from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [4]:
model_name = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [5]:
def load_model(model_name, base = True):
  if base == True:
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype = torch.float16,
      trust_remote_code = True
    ).to(device)

    return model
    
  else:
    bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_quant_type = 'nf4',
      bnb_4bit_compute_dtype = torch.float16,
      bnb_4bit_use_double_quant = True,
    )
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config = bnb_config,
      trust_remote_code = True
    ).to(device)

    return model

In [6]:
model = load_model(model_name, base = False)
model

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((

In [7]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 906070016
Trainable parameters : 100763648
Trainable percentage: 11.12%


## 02 Import Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

## 03 Import Dataset

In [9]:
dataset_name = 'microsoft/orca-math-word-problems-200k'

In [10]:
max_length = 384

In [11]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

README.md:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 200035
})

In [12]:
dataset = dataset.select(range(10000))

In [13]:
dataset.select(range(5)).to_pandas().head()

,question,answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."
2,Dongju selects a piece of paper with a number ...,To find the second smallest and third smallest...
3,"You wanted to subtract 46 from a number, but y...",If you accidentally subtracted 59 instead of 4...
4,The length of one span of Jinseo is about 12 c...,If one span of Jinseo is about 12 centimeters ...


In [14]:
dataset[0]

{'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}

In [15]:
features = list(dataset.features.keys())
print(features)

['question', 'answer']


## 04 Text Formatting

In [16]:
prompt_format = """### Question:\n{}\n### Answer:\n{}"""

In [17]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def preprocess(examples):
  input = examples['question']
  output = examples['answer']
  
  text = prompt_format.format(input, output) + EOS_TOKEN
  return {'prompt' : text}

In [18]:
formatted_dataset = dataset.map(preprocess, remove_columns = features)
formatted_dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt'],
    num_rows: 10000
})

In [19]:
print(formatted_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


## 05 Tokenization

In [20]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [21]:
tokenized_dataset = formatted_dataset.map(tokenize_data)#, batched = True)#, remove_columns = 'text')
tokenized_dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [22]:
print(tokenized_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


In [23]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [24]:
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [25]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,### Question:\nThere is a two-digit natural nu...,"[3757, 15232, 42, 198, 2122, 314, 253, 827, 29...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"### Question:\nIn a big box, there are marbles...","[3757, 15232, 42, 198, 788, 253, 2066, 3985, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"### Question:\nAdam goes to a small school, wh...","[3757, 15232, 42, 198, 31019, 3935, 288, 253, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,### Question:\nLisa is looking to attempt a Wo...,"[3757, 15232, 42, 198, 60, 14765, 314, 3012, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,### Question:\nThere is a rectangular-shaped p...,"[3757, 15232, 42, 198, 2122, 314, 253, 18896, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [26]:
print(train_dataset[0]['prompt'])

### Question:
There is a two-digit natural number whose tens place is 3. Let A and B be the quotient of this number by 10 and the remainder of division by 10, respectively. If B multiplied by 10 plus A is 9 less than A multiplied by 10 plus B, what is the first number?
### Answer:
Let's denote the two-digit number as \( XY \), where \( X \) is the digit in the tens place and \( Y \) is the digit in the ones place. Since the tens place is 3, we have \( X = 3 \).

According to the problem, \( A \) is the quotient of the number by 10, and \( B \) is the remainder of the division by 10. Therefore, \( A = X = 3 \) and \( B = Y \).

The problem states that \( B \times 10 + A \) is 9 less than \( A \times 10 + B \). This can be written as an equation:

\[ B \times 10 + A = A \times 10 + B - 9 \]

Substituting \( A \) and \( B \) with \( 3 \) and \( Y \), respectively, we get:

\[ Y \times 10 + 3 = 3 \times 10 + Y - 9 \]

Simplifying the equation:

\[ 10Y + 3 = 30 + Y - 9 \]

\[ 10Y + 3 = Y + 

In [27]:
print(train_dataset[0]['input_ids'])

[3757, 15232, 42, 198, 2122, 314, 253, 827, 29, 23141, 1782, 1230, 3449, 12281, 1379, 314, 216, 35, 30, 2959, 330, 284, 389, 325, 260, 14498, 1010, 282, 451, 1230, 411, 216, 33, 32, 284, 260, 17867, 282, 7573, 411, 216, 33, 32, 28, 7827, 30, 1094, 389, 25319, 411, 216, 33, 32, 8055, 330, 314, 216, 41, 1181, 670, 330, 25319, 411, 216, 33, 32, 8055, 389, 28, 732, 314, 260, 808, 1230, 47, 198, 3757, 19842, 42, 198, 4239, 506, 25832, 260, 827, 29, 23141, 1230, 347, 3814, 24, 33739, 3814, 643, 837, 3814, 24, 2273, 3814, 25, 314, 260, 11403, 281, 260, 12281, 1379, 284, 3814, 24, 718, 3814, 25, 314, 260, 11403, 281, 260, 2911, 1379, 30, 4311, 260, 12281, 1379, 314, 216, 35, 28, 392, 457, 3814, 24, 2273, 446, 216, 35, 3814, 595, 198, 198, 5449, 288, 260, 1732, 28, 3814, 24, 330, 3814, 25, 314, 260, 14498, 1010, 282, 260, 1230, 411, 216, 33, 32, 28, 284, 3814, 24, 389, 3814, 25, 314, 260, 17867, 282, 260, 7573, 411, 216, 33, 32, 30, 4882, 28, 3814, 24, 330, 446, 2273, 446, 216, 35, 3814, 25, 28

In [28]:
print(train_dataset[0]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 06 Data Collator Set Up

In [29]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 07 Evaluation Metrics Set Up

In [30]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [31]:
torch.cuda.empty_cache()

## 08 Set Up PEFT / LoRA / QLoRA

In [32]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]

#target_modules = ["qkv_proj", "proj_1", "proj_2", "out_proj"]

peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [33]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'LoRA')
peft_model.print_trainable_parameters()

trainable params: 72,351,744 || all params: 1,783,728,128 || trainable%: 4.0562


## 09 Training Model

In [34]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 978421760
Trainable parameters : 72351744
Trainable percentage: 7.39%


In [35]:
torch.cuda.empty_cache()

In [36]:
save_path = './model'

batch_size = 2
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=20,
eval_strategy=steps,
eval_use_gather_object=F

In [37]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

In [38]:
trainer.train()

Currently training with a batch size of: 2
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 200
  Number of trainable parameters = 72,351,744
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more informati

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
20,0.824300,0.824495
40,0.841700,0.791060
60,0.785700,0.749588
80,0.756000,0.712707
100,0.703800,0.695804
120,0.667700,0.688196
140,0.678100,0.683665
160,0.674800,0.681043
180,0.679500,0.680017
200,0.687700,0.679762


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a c

TrainOutput(global_step=200, training_loss=0.7299389839172363, metrics={'train_runtime': 2460.1057, 'train_samples_per_second': 0.65, 'train_steps_per_second': 0.081, 'total_flos': 6471167665766400.0, 'train_loss': 0.7299389839172363, 'epoch': 0.17777777777777778})

## 10 Model Evaluation

In [39]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


Evaluation Results: {'eval_loss': 0.6797617077827454, 'eval_runtime': 51.6304, 'eval_samples_per_second': 3.874, 'eval_steps_per_second': 0.968, 'epoch': 0.17777777777777778}


## 11 Save Model

In [40]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HuggingFaceTB--SmolLM2-1.7B-Instruct/snapshots/1dfd32118ef11b4998e591ed60f2a0e978e15411/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "num_key_value_heads": 32,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 130000,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers.js_config": {
    "kv_cache_dtype": {
      "fp16": "float16",
      "q4f16": "float16"
    }
  },
  "transformers_version": "4.46.3",
  "use_cache": true,

## 12 Load PEFT Model

In [41]:
torch.cuda.empty_cache()

In [42]:
peft_path = save_path + '/LoRA'
peft_path

'./model/LoRA'

In [43]:
peft_model = PeftModel.from_pretrained(model, peft_path)

## 13 Reload & Recheck Base Model

In [44]:
model = load_model(model_name, base = False)
model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HuggingFaceTB--SmolLM2-1.7B-Instruct/snapshots/1dfd32118ef11b4998e591ed60f2a0e978e15411/config.json
Model config LlamaConfig {
  "_name_or_path": "HuggingFaceTB/SmolLM2-1.7B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "num_key_value_heads": 32,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 130000,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers.js_config": {
    "kv_cache_dtype": {
      "fp16": "float16",
      "q4f16": "float16"
    }
  }

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((

In [45]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 906070016
Trainable parameters : 100763648
Trainable percentage: 11.12%


In [46]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 2048, padding_idx=2)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (LoRA): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (LoRA): Linear(in_features=2048, out_features=64, bias=False)
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (LoRA): Linear(in_features=64, out_features=2048, bias=False)
                  (default): Linear(in_features=64, out_features=2048

In [47]:
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1050773504
Trainable parameters : 0
Trainable percentage: 0.00%


## 14 Pre Test & Post Test

In [48]:
def pre_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [49]:
def post_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [50]:
def print_side_by_side(pre_text, post_text, width = 50):
  pre_wrapped = textwrap.wrap(pre_text, width)
  post_wrapped = textwrap.wrap(post_text, width)
    
  print('PRE-TEST'.center(width), ' | ', 'POST-TEST'.center(width))
  print(
    str(sum(p.numel() for p in model.parameters())).center(width),
    '|',
    str(sum(p.numel() for p in peft_model.parameters())).center(width)
  )
  print('=' * width, '|', '=' * width)
    
  for pre, post in zip_longest(pre_wrapped, post_wrapped, fillvalue = ''):
    print(pre.ljust(width), ' | ', post.ljust(width))

In [73]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    906070016                      |                     1050773504                    
================================================== | ==================================================
### Question: There are 40 students in a class. If  |  ### Question: There are 40 students in a class. If
a certain fraction of students are absent, 3/4 of   |  a certain fraction of students are absent, 3/4 of 
the students who are present are in the classroom,  |  the students who are present are in the classroom,
and the rest are in the canteen. There are 9        |  and the rest are in the canteen. There are 9      
students in the canteen. What fraction of students  |  students in the canteen. What fraction of students
are absent? ### Answer: To find the fraction of     |  are absent? ### Answer: To find the fraction of   
students absent, we first need to determine the   

In [65]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    906070016                      |                     1050773504                    
================================================== | ==================================================
### Question: Sehun wants to make a three-digit     |  ### Question: Sehun wants to make a three-digit   
number by using the single-digit natural numbers    |  number by using the single-digit natural numbers  
0, 2, and 9 once each. How many total can he make?  |  0, 2, and 9 once each. How many total can he make?
### Answer: Sehun can make 10 different three-      |  ### Answer: Sehun can make 10 different three-    
digit numbers.  Explanation: 1. First, we need to   |  digit numbers.  Explanation: 1. First, we need to 
find the total number of ways to arrange the        |  find the total number of ways to arrange the      
digits 0, 2, and 9. Since there are three digits, 

In [62]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    906070016                      |                     1050773504                    
================================================== | ==================================================
### Question: If A is the number of diagonals in a  |  ### Question: If A is the number of diagonals in a
heptagon and B is the number of diagonals in an     |  heptagon and B is the number of diagonals in an   
octagon, find the value of B-A. ### Answer: To      |  octagon, find the value of B-A. ### Answer: The   
find the value of B-A, we first need to determine   |  number of diagonals in a heptagon (7 sides) is (7 
the number of diagonals in each polygon.  For a     |  * 6) / 2 = 21. The number of diagonals in an      
heptagon (7 sides): - Number of diagonals =         |  octagon (8 sides) is (8 * 7) / 2 = 28. Therefore, 
n(n-3)/2 - Number of diagonals = 7(7-3)/2 - Number

In [57]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    906070016                      |                     1050773504                    
================================================== | ==================================================
### Question: How many faces does an octahedron     |  ### Question: How many faces does an octahedron   
have? ### Answer: 8 ### Reason: An octahedron is a  |  have? ### Answer: 8 ### Reason: An octahedron is a
polyhedron with eight faces, each of which is a     |  polyhedron with eight faces, each of which is a   
triangle.  Question: How many edges does a          |  triangle.  Question: What is the sum of the       
tetrahedron have? ### Question: How many edges      |  interior angles of a triangle? ### Question: The  
does a tetrahedron have? ### Answer: 4 ### Reason:  |  sum of the interior angles of a triangle is always
A tetrahedron is a polyhedron with four edges.    

In [68]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    906070016                      |                     1050773504                    
================================================== | ==================================================
### Question: Josh has 18 yards of ribbon that is   |  ### Question: Josh has 18 yards of ribbon that is 
to be used equally to some gifts. Each gift will    |  to be used equally to some gifts. Each gift will  
use 2 yards of ribbon. There will be 6 yards of     |  use 2 yards of ribbon. There will be 6 yards of   
ribbon left. How many gifts will receive ribbon?    |  ribbon left. How many gifts will receive ribbon?  
### Answer: To find out how many gifts will         |  ### Answer: To find out how many gifts will       
receive ribbon, we first need to determine how      |  receive ribbon, we first need to determine how    
many yards of ribbon are left after giving some to